<a href="https://colab.research.google.com/github/thistle0420/niconico-videoComment/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
# 動画URL
URL = 'https://www.nicovideo.jp/watch/sm42820659'
# GoogleSpreadsheetのID
KEY = '1G7bizzTjNsyAX4Y3bUuAbq76iGybfYV_AAWy2Y5q7ag'

In [133]:
import requests, random, string
import pandas as pd

def getVideoInfo():
  headers = {
    "X-Frontend-Id": "6",
    "X-Frontend-Version": "0"
  }

  videoId = URL.split('/')[-1]

  actionTrackId = \
    "".join(random.choice(string.ascii_letters + string.digits) for _ in range(10)) \
    + "_" \
    + str(random.randrange(10**(12),10**13))

  url = "https://www.nicovideo.jp/api/watch/v3_guest/{}?actionTrackId={}" \
    .format(videoId, actionTrackId)

  res = requests.post(url, headers=headers).json()

  nvComment = res["data"]["comment"]["nvComment"]
  return nvComment

def getCommentData(nvComment):
  headers = {
    "X-Frontend-Id": "6",
    "X-Frontend-Version": "0",
    "Content-Type": "application/json"
  }

  params = {
    "params": nvComment["params"],
    "additionals": {},
    "threadKey": nvComment["threadKey"]
  }

  url = nvComment["server"] + "/v1/threads"

  res = requests.post(url, json.dumps(params), headers=headers).json()
  return res

def getComments(response, num, fork):
  comments = []
  data = response.json()['data']['threads'][num]
  if bool(data['comments']):
    for comment in data['comments']:
      comments.append([fork, comment['userId'], comment['body'], comment['nicoruCount'], comment['vposMs'], comment['postedAt'],  comment['no']])
  return comments

def getOwner(response):
  return getComments(response, 0, 'owner')

def getMain(response):
  return getComments(response, 1, 'main')

def getEasy(response):
  return getComments(response, 2, 'easy')

nvComment = getVideoInfo()
commentData = getCommentData(nvComment)
threads = commentData['data']['threads']

columns = ['種類','ユーザID','コメント','ニコる','再生時間','書込時間','コメ番']
output = [columns]
for thread in threads:
  fork = thread['fork']
  comments = thread['comments']
  for comment in comments:
    output.append([fork, comment['userId'], comment['body'], comment['nicoruCount'], comment['vposMs'], comment['postedAt'],  comment['no']])

df = pd.DataFrame(output)
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.reset_index(drop=True, inplace=True)
df

,種類,ユーザID,コメント,ニコる,再生時間,書込時間,コメ番
0,main,nvc:1DnIkCkWXyMU5zri_MPiTQrwEFw,うぽつ,0,3410,2023-09-30T18:00:48+09:00,1
1,main,nvc:1yn-5VgPKFK-6YLwXpTbW8JvWuE,うぽつですわ,0,6240,2023-09-30T18:01:33+09:00,2
2,main,nvc:RKlapnh8ZmLAJM6uKRJPTvs2FyQ,うぽつー,0,4980,2023-09-30T18:01:36+09:00,3
3,main,nvc:V60lXP_G1Bkhw4F9VxB24c5Fyy4,うぽつ～,0,9020,2023-09-30T18:01:53+09:00,4
4,main,nvc:RKlapnh8ZmLAJM6uKRJPTvs2FyQ,また麻薬かな？w,0,23740,2023-09-30T18:01:55+09:00,5
...,...,...,...,...,...,...,...
435,easy,nvc:ok8SM2Bsf5mSRO5UTWrF8EdV68Q,88888888,0,85870,2023-10-01T13:14:56+09:00,152
436,easy,nvc:lE_gf_pzpxQg0n312Ojx8rTUd9E,うぽつ,0,0,2023-10-01T13:15:11+09:00,153
437,easy,nvc:lE_gf_pzpxQg0n312Ojx8rTUd9E,うぽつ,0,2570,2023-10-01T13:15:13+09:00,154
438,easy,nvc:Ox-rOfZ0W5R-LEGiD_xkyMSnicU,うぽつ,0,5370,2023-10-01T13:45:03+09:00,155


In [134]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
sh = gc.open_by_key(KEY)
sheetName = URL.split('/')[-1].replace('sm','')

try:
  sh.add_worksheet(sheetName, rows=len(output), cols=len(output[0]))
except Exception as e :
  sh.worksheet(sheetName).clear()

sh.values_append(sheetName, {"valueInputOption": "USER_ENTERED"}, {"values": output})

{'spreadsheetId': '1G7bizzTjNsyAX4Y3bUuAbq76iGybfYV_AAWy2Y5q7ag',
 'updates': {'spreadsheetId': '1G7bizzTjNsyAX4Y3bUuAbq76iGybfYV_AAWy2Y5q7ag',
  'updatedRange': "'42820659'!A1:G441",
  'updatedRows': 441,
  'updatedColumns': 7,
  'updatedCells': 3087}}